In [24]:
import numpy
import pandas
import random
import time

In [25]:
def Split(df, testSize):
    if isinstance(testSize, float):
        testSize = round(testSize * len(df))
    indices = df.index.tolist()
    testIndices = random.sample(population = indices, k = testSize)
    dfTest = df.loc[testIndices]
    dfTrain = df.drop(testIndices)
    return dfTrain, dfTest

def puritycheck(data):
    if len(numpy.unique(data[:, -1])) == 1:
        return True
    else:
        return False

def dataclassification(data):
    uniqueClasses, uniqueClassesCounts = numpy.unique(data[:, -1], return_counts = True)
    return uniqueClasses[uniqueClassesCounts.argmax()]

def PotentialSplit(data, randomAttributes):
    potentialSplits = {}
    _, columns = data.shape
    columnsIndices = list(range(columns - 1))
    if randomAttributes != None  and len(randomAttributes) <= len(columnsIndices):
        columnsIndices = randomAttributes
    for column in columnsIndices:
        values = data[:, column]
        uniqueValues = numpy.unique(values)
        if len(uniqueValues) == 1:
            potentialSplits[column] = uniqueValues
        else:
            potentialSplits[column] = []
            for i in range(len(uniqueValues)):
                if i != 0:
                    currentValue = uniqueValues[i]
                    previousValue = uniqueValues[i - 1]
                    potentialSplits[column].append((currentValue + previousValue) / 2)
    return potentialSplits

def splitData(data, splitColumn, splitValue):
    splitColumnValues = data[:, splitColumn]
    return data[splitColumnValues <= splitValue], data[splitColumnValues > splitValue]

def EntropyCalc(data):
    _, uniqueClassesCounts = numpy.unique(data[:, -1], return_counts = True)
    probabilities = uniqueClassesCounts / uniqueClassesCounts.sum()
    return sum(probabilities * -numpy.log2(probabilities))

def TotalEntropy(dataBelow, dataAbove):
    pDataBelow = len(dataBelow) / (len(dataBelow) + len(dataAbove))
    pDataAbove = len(dataAbove) / (len(dataBelow) + len(dataAbove))
    return pDataBelow * EntropyCalc(dataBelow) + pDataAbove * EntropyCalc(dataAbove)

def BestSplit(data, potentialSplits, randomSplits = None):
    overallEntropy = 9999
    bestSplitColumn = 0
    bestSplitValue = 0
    if randomSplits == None:
        for splitColumn in potentialSplits:
            for splitValue in potentialSplits[splitColumn]:
                dataBelow, dataAbove = splitData(data, splitColumn, splitValue)
                currentOverallEntropy = TotalEntropy(dataBelow, dataAbove)
                if currentOverallEntropy <= overallEntropy:
                    overallEntropy = currentOverallEntropy
                    bestSplitColumn = splitColumn
                    bestSplitValue = splitValue
    else:
        for i in range(randomSplits):
            randomSplitColumn = random.choice(list(potentialSplits))
            randomSplitValue = random.choice(potentialSplits[randomSplitColumn])
            dataBelow, dataAbove = splitData(data, randomSplitColumn, randomSplitValue)
            currentOverallEntropy = TotalEntropy(dataBelow, dataAbove)
            if currentOverallEntropy <= overallEntropy:
                overallEntropy = currentOverallEntropy
                bestSplitColumn = randomSplitColumn
                bestSplitValue = randomSplitValue
    return bestSplitColumn, bestSplitValue

def DecisionTree(dataFrame, currentDepth = 0, minSampleSize = 2, maxDepth = 1000, randomAttributes = None, randomSplits = None):
    if currentDepth == 0:
        global COLUMN_HEADERS
        COLUMN_HEADERS = dataFrame.columns
        data = dataFrame.values
        if randomAttributes != None and randomAttributes <= len(COLUMN_HEADERS) - 1:
            randomAttributes = random.sample(population = list(range(len(COLUMN_HEADERS) - 1)), k = randomAttributes)
        else:
            randomAttributes = None
    else:
        data = dataFrame
    if puritycheck(data) or len(data) < minSampleSize or currentDepth == maxDepth:
        return dataclassification(data)
    else:
        currentDepth += 1
        potentialSplits = PotentialSplit(data, randomAttributes)
        splitColumn, splitValue = BestSplit(data, potentialSplits, randomSplits)
        dataBelow, dataAbove = splitData(data, splitColumn, splitValue)
        if len(dataBelow) == 0 or len(dataAbove) == 0:
            return dataclassification(data)
        else:
            question = str(COLUMN_HEADERS[splitColumn]) + " <= " + str(splitValue)
            decisionSubTree = {question: []}
            yesAnswer = DecisionTree(dataBelow, currentDepth, minSampleSize, maxDepth, randomAttributes, randomSplits)
            noAnswer = DecisionTree(dataAbove, currentDepth, minSampleSize, maxDepth, randomAttributes, randomSplits)
            if yesAnswer == noAnswer:
                decisionSubTree = yesAnswer
            else:
                decisionSubTree[question].append(yesAnswer)
                decisionSubTree[question].append(noAnswer)
            return decisionSubTree

def SampleClassification(sample, decisionTree):
    if not isinstance(decisionTree, dict):
        return decisionTree
    question = list(decisionTree.keys())[0]
    attribute, value = question.split(" <= ")
    if sample[attribute] <= float(value):
        answer = decisionTree[question][0]
    else:
        answer = decisionTree[question][1]
    return SampleClassification(sample, answer)

def PredictionDT(dataFrame, decisionTree):
    predictions = dataFrame.apply(SampleClassification, axis = 1, args = (decisionTree,))
    return predictions

def AccuracyCalculate(predictedResults, category):
    resultCorrect = predictedResults == category
    return resultCorrect.mean()

In [26]:
df = pandas.read_csv("breast_cancer.csv")
df = df.drop("id", axis = 1)
df = df[df.columns.tolist()[1: ] + df.columns.tolist()[0: 1]]
dfTrain, dfTest = Split(df, testSize = 0.25)

In [27]:
print("1804032: Decision Tree")

k = 1
TrainAccuracy = 0
while TrainAccuracy < 100:
    decisionTree = DecisionTree(dfTrain, maxDepth = k)
    decisionTreeTestResults = PredictionDT(dfTest, decisionTree)
    TestAccuracy = AccuracyCalculate(decisionTreeTestResults, dfTest.iloc[:, -1]) * 100
    decisionTreeTrainResults = PredictionDT(dfTrain, decisionTree)
    TrainAccuracy = AccuracyCalculate(decisionTreeTrainResults, dfTrain.iloc[:, -1]) * 100
    print("Depth = {}: ".format(k), end = "")
    print("TestAccuracy = {0:.2f}%, ".format(TestAccuracy), end = "")
    print("TrainAccuracy = {0:.2f}%".format(TrainAccuracy), end = "\n")
    k += 1

1804032: Decision Tree
Depth = 1: TestAccuracy = 90.14%, TrainAccuracy = 92.27%
Depth = 2: TestAccuracy = 90.14%, TrainAccuracy = 92.27%
Depth = 3: TestAccuracy = 95.77%, TrainAccuracy = 96.72%
Depth = 4: TestAccuracy = 95.77%, TrainAccuracy = 97.89%
Depth = 5: TestAccuracy = 96.48%, TrainAccuracy = 99.53%
Depth = 6: TestAccuracy = 95.77%, TrainAccuracy = 100.00%
